In [390]:
import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
import time
import pytz

### Fetch the current weather

In [ ]:

def get_current_weather(city):
    API_KEY = 'a5e8713c6700bfce420cd38b90e2122b'
    BASE_URL = 'https://api.openweathermap.org/data/2.5/weather'
    
    url = f"{BASE_URL}?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)

    if response.status_code != 200:
        raise ValueError(f"API request failed with status {response.status_code}: {response.text}")
    
    data = response.json()

    # Check for 'weather' key
    if 'weather' not in data or not data['weather']:
        raise ValueError("Weather data not found in API responshe.")

    return {
        'city': data.get('name', 'N/A'),
        'current_temp': round(data['main']['temp']),
        'min_temp': data['main']['temp_min'],
        'max_temp': data['main']['temp_max'],
        'humidity': data['main']['humidity'],
        'pressure': data['main'].get('pressure', 1013),
        'wind_speed': data['wind']['speed'],
        'wind_deg': data['wind'].get('deg', 0),
        'Wind_Gust_Speed': data['wind'].get('gust', 0),
        'description': data['weather'][0]['description'],
        'country': data['sys'].get('country', 'N/A')
    }


### read the historical data

In [392]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.dropna()
  df = df.drop_duplicates()

  return df

### data preprocessing

In [393]:
def prepare_data(data):
  le = LabelEncoder()
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  X = data.drop(columns=['RainTomorrow'])
  y = data['RainTomorrow']

  return X,y,le


### Train Rain Prediction Model

In [394]:
def train_rain_model(X,y):
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=42)
  model = RandomForestClassifier()
  model.fit(X_train,y_train)

  y_pred = model.predict(X_test)

  return model

## prepare regression data

In [395]:
def prepare_regression_data(data,features):
    X,y = [],[]
    for i in range(len(data)-1):
        X.append(data[features].iloc[i])

        y.append(data[features].iloc[i+1])
    X = np.array(X).reshape(-1, 1)
    y = np.array(y)

    return X,y

## train th regression model

In [396]:
def train_regression_model(X,y):
    model = RandomForestRegressor()
    model.fit(X,y)

    return model

## predict the future

In [397]:
def predict_future(model,current_value):
    prediction = [current_value]

    for i in range(5):
        next_value = model.predict(np.array([[prediction[-1]]]))

    prediction.append(next_value[0])

    return prediction[1:]

## wheather analysis function

In [ ]:
from datetime import timedelta, datetime
import pytz
import pandas as pd

def weather_view():
    city = input("Enter your city name: ")
    current_weather = get_current_weather(city)

    historical_data = read_historical_data('weather.csv')
    X, y, le = prepare_data(historical_data)
    rain_model = train_rain_model(X, y)

    #   Map wind direction
    wind_deg = current_weather['wind_deg'] % 360
    compass_points = [
        ("N", 0, 11.25), ("NNE", 11.25, 33.75),("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75),("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75), ("N", 348.75, 360)
    ]
    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    #   Safe current data dictionary
    current_data = {
        'MinTemp': current_weather.get('min_temp', 0),
        'MaxTemp': current_weather.get('max_temp', 0),
        'WindGustDir': compass_direction_encoded,
        'WindGustSpeed': current_weather.get('Wind_Gust_Speed', 0),
        'Humidity': current_weather.get('humidity', 0),
        'Pressure': current_weather.get('pressure', 1013),
        'Temp': current_weather.get('current_temp', 0)
    }

    current_df = pd.DataFrame([current_data])
    rain_prediction = rain_model.predict(current_df)[0]

    #   Prepare models for temperature & humidity prediction
    X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

    temp_model = train_regression_model(X_temp, y_temp)
    hum_model = train_regression_model(X_hum, y_hum)

    #   Predict future temperature and humidity
    future_temp = predict_future(temp_model, current_weather['current_temp'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

    #   Format future timestamps
    timezone = pytz.timezone('Asia/Karachi')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
    future_time = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

# Convert NumPy float64 to native float and round to 2 decimal places
    future_temp = [round(float(val), 2) for val in future_temp]
    future_humidity = [round(float(val), 2) for val in future_humidity]


    #   Output results
    print(f"City : {city}, {current_weather['country']}")
    print(f"Current temperature: {current_weather['current_temp']}°C")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")
    print("Future temperature (next hours):", future_temp)
    print("Future humidity (next hours):", future_humidity)

#   Call the function
weather_view()


API Response: {'coord': {'lon': 86.9317, 'lat': 21.4942}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 28.55, 'feels_like': 32.79, 'temp_min': 28.55, 'temp_max': 28.55, 'pressure': 1000, 'humidity': 76, 'sea_level': 1000, 'grnd_level': 998}, 'visibility': 10000, 'wind': {'speed': 4.62, 'deg': 228, 'gust': 9.57}, 'clouds': {'all': 100}, 'dt': 1753635307, 'sys': {'country': 'IN', 'sunrise': 1753573396, 'sunset': 1753620843}, 'timezone': 19800, 'id': 1277599, 'name': 'Balasore', 'cod': 200}
City : balasore, IN
Current temperature: 29°C
Rain Prediction: Yes
Future temperature (next hours): [21.91]
Future humidity (next hours): [60.94]
